In [1]:
from utils.imports import *

/home/leonard/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_net(input_shape, load_weight_path=None, features=False, mal=False):
    width = 64
    inputs = Input(shape=(1, 32, 32, 32), name="input_1")
    x = inputs
    x = AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), border_mode="same")(x)
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width, 3, 3, 3, activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1')(x)

    # 2nd layer group
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*2, 3, 3, 3, activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool2')(x)
    x = Dropout(p=0.3)(x)

    # 3rd layer group
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1))(x)
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool3')(x)
    x = Dropout(p=0.4)(x)

    # 4th layer group
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4a', subsample=(1, 1, 1))(x)
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1, 1),)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool4')(x)
    x = Dropout(p=0.5)(x)
    print (x)
       
    last64 = Convolution3D(64, 2, 2, 2, activation="relu", name="last_64")(x)
    out_class = Convolution3D(1, 1, 1, 1, activation="softmax", name="out_class_last")(last64)
    out_class = Flatten(name="out_class")(x)
    
    out_class = Dense(2)(out_class) 
    #out_class = BatchNormalization(axis = 1)(out_class)
    out_class = Activation('softmax')(out_class)
    
    model = Model(input=inputs, output=out_class)
    model.compile(optimizer=SGD(lr=1e-3, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])   
    return model

In [3]:
output_path = PATH['cls_train_cube_30']
output_true = PATH['vgg_cls_train_cube_30_true']
output_false = PATH['vgg_cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [4]:
def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    train_nb = 15000   
    file_list_true = get_dirfiles(output_true)
    file_list_false = get_dirfiles(output_false)        
    file_list_true = np.random.choice(file_list_true, train_nb)
              
    file_list_false = np.random.choice(file_list_false, train_nb)    
    nb_true = len(file_list_true) + len(file_list_false) 
    print nb_true

#     pd.read_csv(nb_true, skiprows=(i for i in range(160000) if i % 5 == 0))

    
    sample = np.zeros([nb_true,32, 32, 32])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, load_check = False,batch_size=2, epochs=20,check_name = None):
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 10, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = classifier((1, 32, 32, 32),128,(3, 3, 3), (2, 2, 2))
        model = get_net((1, 32, 32, 32))
    x,y = train_generator(output_true,output_false)
    model.fit(x,y, batch_size=batch_size, epochs=epochs,
              validation_split=0.2,verbose=1, callbacks=callbacks, shuffle=True) 
    return model

In [5]:
fenlei_fit('Fenge_32_32_32_0704', load_check = False, batch_size=4, epochs=100, check_name = 'Fenge_32_32_32_0703')

Tensor("dropout_3/cond/Merge:0", shape=(?, 512, 2, 2, 2), dtype=float32)


  0%|          | 6/15000 [00:00<05:10, 48.36it/s]

30000


100%|██████████| 15000/15000 [03:41<00:00, 67.67it/s]


Train on 24000 samples, validate on 6000 samples
Epoch 1/100
24000/24000 [==============================] - 199s 8ms/step - loss: 0.5902 - acc: 0.6646 - val_loss: 0.6346 - val_acc: 0.6860
Epoch 2/100
24000/24000 [==============================] - 192s 8ms/step - loss: 0.3472 - acc: 0.8450 - val_loss: 0.2515 - val_acc: 0.9272
Epoch 3/100
24000/24000 [==============================] - 193s 8ms/step - loss: 0.2183 - acc: 0.9104 - val_loss: 0.1280 - val_acc: 0.9723
Epoch 4/100
24000/24000 [==============================] - 193s 8ms/step - loss: 0.1661 - acc: 0.9357 - val_loss: 0.1799 - val_acc: 0.9467
Epoch 5/100
24000/24000 [==============================] - 193s 8ms/step - loss: 0.1313 - acc: 0.9510 - val_loss: 0.2468 - val_acc: 0.9207
Epoch 6/100
24000/24000 [==============================] - 193s 8ms/step - loss: 0.1031 - acc: 0.9627 - val_loss: 0.1969 - val_acc: 0.9418
Epoch 7/100
24000/24000 [==============================] - 194s 8ms/step - loss: 0.0850 - acc: 0.9679 - val_loss: 0.2

In [6]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [7]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_32_32_32_0704.h5')

In [8]:
file_list_true = np.random.choice(file_list_true, 1000)
file_list_false = np.random.choice(file_list_false, 1000)

In [9]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

0.945


In [10]:
cc

[array([[9.9999905e-01, 8.9850209e-07]], dtype=float32),
 array([[9.9997973e-01, 2.0281863e-05]], dtype=float32),
 array([[1.000000e+00, 4.748252e-09]], dtype=float32),
 array([[9.9996984e-01, 3.0124003e-05]], dtype=float32),
 array([[9.9998796e-01, 1.2076364e-05]], dtype=float32),
 array([[0.9984988 , 0.00150126]], dtype=float32),
 array([[9.9999714e-01, 2.8895886e-06]], dtype=float32),
 array([[0.9979977 , 0.00200227]], dtype=float32),
 array([[9.9998772e-01, 1.2310754e-05]], dtype=float32),
 array([[9.9988186e-01, 1.1817517e-04]], dtype=float32),
 array([[1.000000e+00, 3.231063e-08]], dtype=float32),
 array([[9.9999869e-01, 1.3681987e-06]], dtype=float32),
 array([[1.0000000e+00, 1.2084677e-09]], dtype=float32),
 array([[9.999999e-01, 1.318523e-07]], dtype=float32),
 array([[9.999908e-01, 9.224860e-06]], dtype=float32),
 array([[0.9987186 , 0.00128139]], dtype=float32),
 array([[0.00867576, 0.9913243 ]], dtype=float32),
 array([[9.999995e-01, 5.222353e-07]], dtype=float32),
 array([

In [11]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0

In [12]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

0.86


In [13]:
cc

[array([[0.0013843 , 0.99861574]], dtype=float32),
 array([[0.6706076 , 0.32939237]], dtype=float32),
 array([[0.810005  , 0.18999499]], dtype=float32),
 array([[2.3518382e-04, 9.9976486e-01]], dtype=float32),
 array([[1.075866e-05, 9.999893e-01]], dtype=float32),
 array([[9.4822544e-04, 9.9905175e-01]], dtype=float32),
 array([[5.374441e-10, 1.000000e+00]], dtype=float32),
 array([[0.7856297 , 0.21437036]], dtype=float32),
 array([[3.45856e-15, 1.00000e+00]], dtype=float32),
 array([[0.00487428, 0.9951257 ]], dtype=float32),
 array([[6.445142e-07, 9.999994e-01]], dtype=float32),
 array([[3.1877964e-04, 9.9968123e-01]], dtype=float32),
 array([[1.1449815e-13, 1.0000000e+00]], dtype=float32),
 array([[3.9630532e-10, 1.0000000e+00]], dtype=float32),
 array([[7.3426614e-05, 9.9992657e-01]], dtype=float32),
 array([[0.6199517, 0.3800483]], dtype=float32),
 array([[0.00229147, 0.99770856]], dtype=float32),
 array([[4.0093804e-09, 1.0000000e+00]], dtype=float32),
 array([[7.6836565e-10, 1.00

### def solo(input_shape, load_weight_path=None, features=False, mal=False):
    width = 64
    inputs = Input(shape=(1, 32, 32, 32), name="input_1")
    x = inputs
    x = AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), border_mode="same")(x)
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width, 3, 3, 3, activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1')(x)

    # 2nd layer group
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*2, 3, 3, 3, activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool2')(x)
    x = Dropout(p=0.3)(x)

    # 3rd layer group
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1))(x)
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool3')(x)
    x = Dropout(p=0.4)(x)

    # 4th layer group
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4a', subsample=(1, 1, 1))(x)
    #x = BatchNormalization(axis = 1)(x)
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1, 1),)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool4')(x)
    x = Dropout(p=0.5)(x)
       
    last64 = Convolution3D(64, 2, 2, 2, activation="relu", name="last_64")(x)
    out_class = Convolution3D(1, 1, 1, 1, activation="softmax", name="out_class_last")(last64)
    out_class = Flatten(name="out_class")(x)
    
    out_class = Dense(2)(out_class) 
    #out_class = BatchNormalization(axis = 1)(out_class)
    out_class = Activation('softmax')(out_class)
    
    model = Model(input=inputs, output=out_class)
    model.compile(optimizer=SGD(lr=1e-3, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) 
    plot_model(model, to_file='model1.png',show_shapes=True, show_layer_names=False)
    return model

solo((1, 32, 32, 32))